## CNN classification

- 1 region, all info about speakers, 4 sessions, white noise, 50 info per neuron (per speaker), 100 neurons per region => image 100x250(5x50) - 8 of them (2 states x 4 sessions) x 30 (sampling points over time)
- final 240 images (120 per state)
- oversampling input images with NaNs (maybe random oversampling)
- combination of 2 different regions in one image/mice???
- including bandpass noise into data representation 
- representing input images as different stimulus (100 x 50)
---
- training on combined sessions, different sessions (evaluating results on different sessions)
- maybe training on the particular stimulus and then seeing wich stimulus is the most inforative for the state classification
- performing classification on different regions to see which region (to see if some regions have very noninformative responses)
- performin classification on different combination of regions???
- comparing the classificator performance - input data is white noise/bandpass noise/combination of noises
---
- stimulus classificators

### 1. Classification between anest and awake state